In [ ]:
import os
import gc
import sys
import time
import pickle
import random
import numpy as np
import pandas as pd
import datatable as dt
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
 
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff

from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

import lightgbm as lgb

from sklearn.metrics import roc_auc_score, mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import (StandardScaler, PowerTransformer,
                                   QuantileTransformer ,LabelEncoder, 
                                   OneHotEncoder,OrdinalEncoder)

In [ ]:
folder_path = '../input/tabular-playground-series-may-2021'
train_data = pd.read_csv(f'{folder_path}/train.csv')
test_data = pd.read_csv(f'{folder_path}/test.csv')
sample = pd.read_csv(f'{folder_path}/sample_submission.csv')

In [ ]:
train_data.head()

In [ ]:
features = [f'feature_{i}' for i in range(50)]

target = train_data['target']
le = LabelEncoder()
target = le.fit_transform(target)

train_data = train_data[features].to_numpy()
test_data = test_data[features].to_numpy()

scaler = StandardScaler()
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)

## LGBM

In [ ]:
nfolds = 5
seed = 43

# params = {
#  'objective':'multiclass',
#  'num_class':4,
#  'reg_alpha': 6.147694913504962,
#  'reg_lambda': 0.002457826062076097,
#  'colsample_bytree': 0.3,
#  'subsample': 0.8,
#  'learning_rate': 0.001,
#  'max_depth': 20,
#  'num_leaves': 111,
#  'min_child_samples': 285,
#  'random_state': 48,
# 'verbose':-1,
#  'n_estimators': 1600000,
# #  'n_estimators': 100,
#  'cat_smooth': 39}

params = {
 'objective':'multiclass',
 'num_class':4,
 'random_state': 43,
'verbose':-1,
 'n_estimators': 10000}

In [ ]:
lgbm_preds = np.zeros((test_data.shape[0],4))

kfold = StratifiedKFold(n_splits=nfolds,shuffle=True,random_state=seed)
lgbm_scores = list()
for k, (train_idx, valid_idx) in enumerate(kfold.split(X=train_data,y=target)):
    print(f"Fold: {k}")
    lgb_train = lgb.Dataset(train_data[train_idx],target[train_idx])
    lgb_valid = lgb.Dataset(train_data[valid_idx],target[valid_idx],reference=lgb_train)
    lgb_model = lgb.train(params,
                      lgb_train, 
                      valid_sets=[lgb_train,lgb_valid],
                      verbose_eval=1000,
                      early_stopping_rounds=800,
                      )
    lgbm_preds += lgb_model.predict(test_data)/nfolds

In [ ]:
sample.loc[:,['Class_1','Class_2','Class_3','Class_4']] = lgbm_preds
sample.to_csv("submission.csv",index=False)
sample.head()